In [3]:
# !pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 4.5 MB/s eta 0:00:00a 0:00:01


In [1]:
def count_and_send_back(key, value):
    print("1")
    import ast
    from pyspark import SparkContext
    from Kafka_Helpers import Producer
    
    # Create Spark context
    sc = SparkContext.getOrCreate()
    
    # make list of lists out of string
    l = ast.literal_eval(value)
    
    # flatten list of lists to one single list
    l = [item for sublist in l for item in sublist]
    
    # create Spark RDD from flattened list
    wordsRDD = sc.parallelize(l, 4)
    
    # count
    res = wordsRDD.map(lambda w: (w, 1)).reduceByKey(lambda x,y: x+y).collect()
    
    # send result (unique words + count) back to another Kafka topic
    p = Producer(server='kafka', port=9092)
    p.send('topic_d', 'mykey', res)
    print(res)
    
def print_key_value(key, value):
    print(f'{key}:{value}')
    

In [2]:
from Kafka_Helpers import Consumer, Producer

Consumer(server='kafka', port=9092, topic_name='topic_c', handler=count_and_send_back)
# Consumer(server='kafka', port=9092, topic_name='topic_d', handler=print_key_value)

Waiting for new events...
